# **Programming Assignment 2** : Problem 1

## **Collaborative Filtering: Recommendation Systems**

#### Loading data from GitHub Repo

In [1]:
!wget https://github.com/gagan-iitb/DS-503/raw/main/Assignments/Assignment2-CollabFiltering/Test_set.csv
!wget https://github.com/gagan-iitb/DS-503/raw/main/Assignments/Assignment2-CollabFiltering/Training_set.csv
!wget https://github.com/gagan-iitb/DS-503/raw/main/Assignments/Assignment2-CollabFiltering/Validation_set.csv
!wget https://github.com/gagan-iitb/DS-503/raw/main/Assignments/Assignment2-CollabFiltering/Validation_set_with_groundtruth.csv

--2021-09-26 16:31:58--  https://github.com/gagan-iitb/DS-503/raw/main/Assignments/Assignment2-CollabFiltering/Test_set.csv
Resolving github.com (github.com)... 140.82.113.4
Connecting to github.com (github.com)|140.82.113.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/gagan-iitb/DS-503/main/Assignments/Assignment2-CollabFiltering/Test_set.csv [following]
--2021-09-26 16:31:58--  https://raw.githubusercontent.com/gagan-iitb/DS-503/main/Assignments/Assignment2-CollabFiltering/Test_set.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 744265 (727K) [text/plain]
Saving to: ‘Test_set.csv’

Test_set.csv        100%[===================>] 726.82K  --.-KB/s    in 0.03s   

2021-09-26 16:31:59 (25.

### Importing required packages

In [2]:
import numpy as np
import pandas as pd
import seaborn as sn

#### Loading the Taining Dataset with `Unnamed: 0` as index

In [4]:
df_train = pd.read_csv('Training_set.csv', index_col='Unnamed: 0')
df_train

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,6.75,8.64,4.42,7.43,4.56,-0.97,4.66,-0.68,3.30,-1.21,0.87,8.64,8.35,9.17,0.05,7.57,4.71,0.87,-0.39,6.99,6.50,-0.92,7.14,9.03,-1.80,0.73,7.09,3.40,-0.87,7.91,...,-6.70,-3.35,-9.03,4.47,4.08,-3.83,8.74,1.12,0.78,7.52,-5.00,2.77,8.30,7.77,7.33,6.21,7.72,8.98,8.64,8.20,3.93,4.85,4.85,6.07,8.98,4.51,-0.05,3.69,4.56,0.58,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
1,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,99.00,99.00,7.33,7.57,9.37,6.17,-6.36,-6.89,-7.86,9.03,9.03,9.03,7.28,99.00,8.25,99.00,99.00,7.48,7.28,7.28,8.93,99.00,6.17,7.28,99.00,99.00,8.98,7.33,99.00,6.17,9.08,7.33,...,6.46,7.28,99.00,99.00,7.04,7.28,99.00,7.28,8.25,99.00,99.00,99.00,99.00,99.00,99.00,8.93,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,9.03,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
2,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,99.00,1.84,7.33,6.60,6.31,8.11,-7.23,-6.65,1.17,-6.60,-3.64,-2.09,5.34,99.00,99.00,99.00,99.00,2.91,3.93,6.75,6.60,99.00,6.65,-6.12,99.00,7.57,6.21,6.65,99.00,-8.30,7.18,2.82,...,0.00,-3.69,99.00,99.00,7.82,0.24,99.00,7.28,-2.33,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,0.63,99.00,99.00,-2.33,99.00,99.00,99.00,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00
3,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,8.25,6.84,-3.93,7.23,-2.33,-9.66,2.72,-1.36,2.57,4.51,8.20,6.12,8.30,-1.26,7.77,1.89,-1.17,5.68,8.45,4.61,8.06,-9.47,7.28,5.68,2.48,3.20,-1.26,6.80,4.51,2.48,...,7.38,6.17,4.71,-2.28,7.38,4.56,7.14,4.22,3.01,3.83,99.00,99.00,99.00,99.00,99.00,99.00,4.13,99.00,99.00,99.00,5.24,5.92,0.87,7.28,3.93,-0.63,6.31,4.71,2.82,2.96,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60
4,-6.17,-3.54,0.44,-8.50,-7.09,-4.32,-8.69,-0.87,-6.65,-1.80,-6.80,-5.73,-5.00,-8.59,0.49,-8.93,-3.69,-2.18,-2.28,-6.12,-3.01,-0.58,-2.38,-7.77,-2.23,-2.28,-5.24,-5.53,-0.68,-6.94,-3.93,-0.34,-6.60,-5.49,-1.55,-0.49,-6.21,-8.06,-5.39,-7.04,...,-6.65,-2.86,-4.61,-9.42,-7.91,-9.81,-9.42,-8.98,-1.94,-6.99,-9.66,-5.19,-5.00,-4.42,-7.28,-6.50,-4.32,-6.94,-9.85,0.73,-6.21,-2.86,-7.62,-6.12,-0.29,-1.41,-3.93,-1.94,-5.68,-4.71,-3.54,-6.89,-0.68,-2.96,-2.18,-3.35,0.05,-9.08,-5.05,-3.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,99.00,99.00,99.00,99.00,-7.86,99.00,-9.81,6.21,99.00,99.00,99.00,99.00,-7.67,99.00,-7.86,-0.39,-7.33,6.12,-8.50,-0.39,-8.93,99.00,-6.94,99.00,99.00,-6.89,3.59,3.35,-7.82,99.00,-7.82,6.41,99.00,-6.89,3.64,-7.48,99.00,99.00,-6.89,99.00,...,-4.66,-0.39,99.00,99.00,-0.58,-7.62,99.00,-6.80,1.60,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,0.73,99.00,99.00,99.00,99.00,99.00,-0.49,99.00,99.00,99.00,4.22,99.00
19996,7.57,8.93,1.55,3.06,0.49,2.18,8.98,1.60,-0.49,8.79,6.94,9.03,-0.44,8.79,5.68,-1.07,-0.24,8.25,2.23,2.23,8.59,8.79,8.93,8.93,3.93,8.79,9.03,-1.02,9.13,-0.24,8.01,6.02,4.03,8.83,8.20,8.35,9.13,8.88,1.50,-1.31,...,8.54,-0.63,3.79,-0.49,5.73,-1.02,-0.10,4.42,7.91,3.54,8.59,6.12,0.10,-6.36,6.41,9.03,2.14,2.14,2.14,2.09,8.88,1.07,8.88,9.22,3.35,3.35,8.30,5.63,8.93,2.62,8.83,8.83,8.83,0.00,-1.31,9.03,9.08,-0.78,-2.72,2.52
19997,8.93,-9.17,-4.66,-9.71,8.83,8.88,8.35,8.54,4.85,3.79,1.31,4.66,-4.42,5.58,-2.86,-2.57,2.04,7.52,1.50,8.98,1.26,3.83,5.63,4.85,9.03,5.68,8.74,9.17,9.17,0.39,2.09,-0.83,-9.56,6.21,8.93,8.40,2.91,5.73,8.88,8.83,...,7.82,8.69,-8.35,0.87,3.20,8.45,2.96,8.79,3.45,4.85,5.19,6.75,

#### Loading the Test Dataset with `Unnamed: 0` as index

In [5]:
df_test = pd.read_csv('Test_set.csv', index_col='Unnamed: 0')
df_test

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
20001,99.00,6.60,99.0,99.00,99.00,99.00,99.0,99.00,99.00,99.00,99.00,99.00,99.00,6.21,99.00,99.00,99.00,1.60,4.13,99.00,99.00,99.00,-3.83,99.00,99.00,1.7,6.41,7.09,99.00,2.86,99.00,5.44,99.00,2.52,5.78,99.00,99.0,99.00,99.00,99.00,...,99.00,99.00,3.79,99.00,99.00,99.00,99.00,99.00,5.58,99.00,99.0,99.00,99.00,99.00,99.00,99.00,99.0,99.00,99.00,99.00,99.0,99.0,99.00,99.0,99.0,99.00,99.0,99.00,99.00,99.0,99.00,99.00,99.00,99.00,99.0,99.00,99.0,99.00,99.00,99.00
20002,99.00,99.00,99.0,99.00,99.00,99.00,99.0,-2.48,99.00,99.00,99.00,99.00,2.48,99.00,0.53,99.00,99.00,-5.24,-7.77,-7.86,99.00,99.00,99.00,2.72,6.50,99.0,99.00,99.00,99.00,99.00,8.69,99.00,6.36,99.00,99.00,99.00,99.0,99.00,99.00,1.80,...,99.00,99.00,99.00,99.00,99.00,99.00,2.67,99.00,99.00,99.00,99.0,99.00,8.93,2.62,99.00,8.93,99.0,7.62,9.03,99.00,99.0,99.0,7.57,99.0,99.0,-5.49,99.0,-0.78,99.00,99.0,-0.49,-7.18,99.00,-9.17,99.0,-7.96,99.0,3.50,99.00,7.96
20003,99.00,99.00,99.0,99.00,99.00,-1.60,99.0,8.64,99.00,99.00,5.24,99.00,1.21,2.52,99.00,99.00,99.00,8.79,4.81,2.77,99.00,99.00,99.00,2.23,99.00,99.0,99.00,9.27,99.00,99.00,99.00,99.00,99.00,99.00,8.59,99.00,99.0,6.31,5.05,8.98,...,99.00,99.00,99.00,99.00,9.27,99.00,99.00,99.00,99.00,99.00,99.0,99.00,99.00,99.00,99.00,99.00,99.0,99.00,99.00,99.00,99.0,99.0,99.00,99.0,99.0,99.00,99.0,99.00,99.00,99.0,99.00,99.00,99.00,99.00,99.0,99.00,99.0,99.00,99.00,99.00
20004,99.00,5.00,99.0,99.00,99.00,99.00,99.0,99.00,99.00,99.00,99.00,3.98,1.50,99.00,-1.12,3.35,99.00,-2.14,99.00,-4.42,99.00,99.00,99.00,1.60,1.12,99.0,2.72,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.0,1.17,99.00,99.00,...,99.00,99.00,99.00,99.00,99.00,5.24,99.00,7.52,99.00,99.00,99.0,99.00,99.00,99.00,99.00,99.00,99.0,99.00,99.00,99.00,99.0,99.0,99.00,99.0,99.0,1.50,99.0,99.00,99.00,99.0,99.00,99.00,99.00,99.00,99.0,99.00,99.0,99.00,99.00,99.00
20005,99.00,99.00,99.0,99.00,-0.05,99.00,99.0,99.00,99.00,99.00,99.00,99.00,-3.79,99.00,99.00,99.00,99.00,0.63,-0.92,99.00,99.00,99.00,99.00,99.00,99.00,99.0,3.74,99.00,5.15,99.00,99.00,2.09,99.00,99.00,99.00,-5.63,99.0,99.00,99.00,99.00,...,99.00,-6.41,99.00,99.00,-0.44,-0.19,99.00,99.00,99.00,99.00,99.0,99.00,99.00,99.00,99.00,99.00,99.0,99.00,99.00,99.00,99.0,99.0,99.00,99.0,99.0,99.00,99.0,99.00,99.00,99.0,99.00,99.00,99.00,99.00,99.0,99.00,99.0,99.00,99.00,99.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21996,3.59,0.92,99.0,99.00,99.00,99.00,99.0,99.00,-4.22,99.00,99.00,99.00,99.00,99.00,1.21,-4.47,99.00,99.00,99.00,99.00,1.36,3.83,3.59,99.00,99.00,99.0,99.00,99.00,4.13,99.00,99.00,2.52,99.00,99.00,0.68,99.00,99.0,99.00,2.67,99.00,...,99.00,99.00,99.00,3.25,99.00,2.33,99.00,99.00,99.00,99.00,99.0,5.92,99.00,99.00,6.07,99.00,99.0,99.00,99.00,3.16,99.0,99.0,99.00,99.0,99.0,99.00,99.0,99.00,7.09,99.0,99.00,2.86,6.70,-3.30,99.0,99.00,99.0,4.61,4.37,99.00
21997,99.00,99.00,99.0,99.00,99.00,1.41,99.0,-2.67,99.00,99.00,99.00,99.00,99.00,99.00,-3.54,99.00,99.00,99.00,99.00,99.00,-3.45,99.00,99.00,99.00,99.00,99.0,99.00,5.78,99.00,99.00,-1.84,99.00,99.00,99.00,99.00,8.69,99.0,99.00,99.00,99.00,...,99.00,8.54,8.20,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.0,99.00,99.00,99.00,99.00,99.00,99.0,99.00,99.00,99.00,99.0,99.0,99.00,99.0,99.0,99.00,99.0,99.00,99.00,99.0,99.00,99.00,99.00,99.00,99.0,99.00,99.0,99.00,99.00,99.00
21998,99.00,99.00,1.5,99.00,99.00,1.84,99.0,99.00,-4.08,-3.01,99.00,99.00,99.00,0.83,99.00,99.00,99.00,2.38,-9.71,99.00,99.00,-3.74,1.60,99.00,99.00,99.0,0.19,99.00,-1.70,2.82,99.00,99.0

### **Question 1 :** Pseudo-Inverse Approach

Assume that we have to guess the rating of an item for a given user X. We know the rating of 30 items for this user (in our validation or test set). Now, we take all the users Y, who have rated all 100 items. We estimate that the ratings of user X are a linear weighted combination of the ratings of these users Y. So, we have an under-determined system with 30 equations and Y variables. Using the pseudo-inverse approach, we can compute the least-square estimate of the weights. Using these weights, we can estimate the ratings of all the 70 items for user X as weighted combinations of the ratings of users Y. 


Now, at first, we check each user whether that user have given all 100 ratings or not.<br> If we found all ratings, then we append the index of that user in the `good_train_users` list,<br> otherwise we append the index in `bad_train_users` list

In [6]:
good_train_users = []
bad_train_users = []

for i in range(df_train.shape[0]):
  if (df_train.iloc[i] < 99).all(): good_train_users.append(i)
  else: bad_train_users.append(i)

print(len(good_train_users))
print(len(bad_train_users))

5736
14264


So, finally we get 5736 users having all 100 ratings. (Set Y according to the question)

Now, we have to find one user from test set, who have 30 valid ratings. (User X)

In [7]:
sample_test_user_index = -1

for i in range(df_test.shape[0]):
  if sum(df_test.iloc[i] < 99) == 30:
    sample_test_user_index = i
    break

print('Index =', sample_test_user_index)
df_test.iloc[sample_test_user_index].value_counts()

Index = 1


 99.00    70
 8.93      2
-0.49      1
 6.50      1
 3.50      1
-0.78      1
 2.72      1
-5.49      1
 5.63      1
 7.62      1
 2.62      1
 7.18      1
-5.24      1
 0.53      1
 7.57      1
 2.48      1
 9.03      1
-9.17      1
-3.79      1
-7.77      1
 1.80      1
 2.67      1
-7.96      1
 8.69      1
 6.36      1
 7.09      1
-2.48      1
 7.96      1
-7.18      1
-7.86      1
Name: 20002, dtype: int64

So, we get an user index = `1`, who have given exactly 30 reviews

Now, we are capturing the column indices for which, the user X have given reviews<br> The list `good_columns` contains all column indices having valid rating and<br> `bad_columns` have the column indices with null value.

In [8]:
good_columns = []
bad_columns = []

for i in range(100):
  if df_test.iloc[sample_test_user_index][i] == 99.0:
    bad_columns.append(str(i+1))
  else: good_columns.append(str(i+1))

print(good_columns)
print(bad_columns)

['8', '13', '15', '18', '19', '20', '24', '25', '31', '33', '40', '44', '47', '48', '59', '67', '73', '74', '76', '78', '79', '83', '86', '88', '91', '92', '94', '96', '98', '100']
['1', '2', '3', '4', '5', '6', '7', '9', '10', '11', '12', '14', '16', '17', '21', '22', '23', '26', '27', '28', '29', '30', '32', '34', '35', '36', '37', '38', '39', '41', '42', '43', '45', '46', '49', '50', '51', '52', '53', '54', '55', '56', '57', '58', '60', '61', '62', '63', '64', '65', '66', '68', '69', '70', '71', '72', '75', '77', '80', '81', '82', '84', '85', '87', '89', '90', '93', '95', '97', '99']


Now, we are dropping all `bad_train_users` as well as the `bad_columns` from a copy of train dataset.<br> So now we have a 5736 x 30 dataset as `df_train_trimmed`. And finally, we are converting<br>the dataset into numpy matrix `X` for easy manipulation.

In [9]:
df_train_trimmed = df_train.copy()
df_train_trimmed.drop(bad_train_users, axis=0, inplace=True)
df_train_trimmed.drop(bad_columns, axis=1, inplace=True)
X = np.matrix(df_train_trimmed).T
print(X.shape)
X

(30, 5736)


matrix([[-5.34, -0.87,  1.41, ...,  1.6 ,  8.54, -5.49],
        [ 4.42, -5.  , -6.94, ..., -0.44, -4.42, -5.05],
        [ 4.56,  0.49,  0.29, ...,  5.68, -2.86, -5.  ],
        ...,
        [-2.14, -3.35, -3.35, ...,  9.03,  8.35,  0.92],
        [ 0.34, -9.08, -0.05, ..., -0.78,  0.39,  6.07],
        [ 1.07, -3.45,  0.  , ...,  2.52,  4.47,  6.5 ]])

And, our matrix of size 30 x 1 `Y` is the sample user, who have only 30 ratings.

In [10]:
Y = np.matrix(df_test.iloc[sample_test_user_index].drop(bad_columns)).reshape((30, 1))
Y

matrix([[-2.48],
        [ 2.48],
        [ 0.53],
        [-5.24],
        [-7.77],
        [-7.86],
        [ 2.72],
        [ 6.5 ],
        [ 8.69],
        [ 6.36],
        [ 1.8 ],
        [-3.79],
        [ 5.63],
        [ 7.09],
        [ 7.18],
        [ 2.67],
        [ 8.93],
        [ 2.62],
        [ 8.93],
        [ 7.62],
        [ 9.03],
        [ 7.57],
        [-5.49],
        [-0.78],
        [-0.49],
        [-7.18],
        [-9.17],
        [-7.96],
        [ 3.5 ],
        [ 7.96]])

Now, we can say

    X . W = Y
    W = (X)-1 . Y

So, to get the weight matrix, we first have to inverse our matrix `X`<br> Next, we have to perform dot product of `X_inv` and `Y`<br> Here, we are using `np.linalg.pinv()` to calculate the pseudoinverse of the matrix `X`


In [11]:
X_inv = np.linalg.pinv(X)
weights = X_inv.dot(Y)
weights.shape

(5736, 1)

So, finally we get all the weights required to predict the behaviour of all 5736 users<br>Now, again we are applying these rules to the rest 70 columns to predict those ratings for the test user.<br>
So, we are again copying the train dataser and dropping the bad users and good columns<br>So finally we get a 3769 x 70 dataset, and we will multiply it wit the `weught` matrix to get a result of size 70 x 1<br>These results are the predicted ratings for the test user

In [12]:
df_recover_data = df_train.copy()
df_recover_data.drop(bad_train_users, axis=0, inplace=True)
df_recover_data.drop(good_columns, axis=1, inplace=True)
recovery_matrix = np.matrix(df_recover_data).T
recovery_matrix.shape

(70, 5736)

In [13]:
recovery_matrix.dot(weights)

matrix([[ 2.47297847],
        [ 5.03106487],
        [ 4.95562266],
        [ 4.53723416],
        [ 1.71936273],
        [ 5.17485733],
        [-1.43001711],
        [ 0.12801288],
        [ 1.51554309],
        [ 6.11315592],
        [ 6.40794654],
        [ 7.03030582],
        [-4.80273808],
        [ 2.82483559],
        [ 6.11502005],
        [ 4.27292108],
        [ 1.70101511],
        [ 6.62158323],
        [ 5.81050792],
        [ 1.54953084],
        [ 4.76957233],
        [ 3.89054873],
        [ 2.30614751],
        [ 6.83064624],
        [ 3.79801351],
        [ 3.26119743],
        [ 0.7779466 ],
        [ 1.5338452 ],
        [ 5.96707046],
        [ 3.9484177 ],
        [ 5.82706569],
        [ 1.11183449],
        [ 3.86133942],
        [ 5.05738081],
        [ 3.51656257],
        [ 3.25217914],
        [ 9.01421805],
        [ 7.60869775],
        [ 4.32624478],
        [ 5.20378571],
        [ 2.15455349],
        [ 4.31509673],
        [-1.77911587],
        [-1

### **Question 2 :** Nearest Neighbor approach: Item-Item Collaborative Filtering

In this approach, the rating of an item is estimated using the ratings of similar items. Assume that we have to guess the rating of an item “i” for a given user X. We first find K (e.g K=3) other items that have been rated by X and are most similar to item “i”. Similarity can be computed using the Pearson Correlation Coefficient (take dot product after subtracting the mean). Using the correlation coefficients as weights, we can estimate the rating as the weighted average of the known ratings of items by that user. We also
adjust for the deviation of the average ratings of the item “i” and user “X” from the overall average rating of all the items in the dataset.<br><br>
You can read section 9.3 of the MMDS book or<br>
https://towardsdatascience.com/comprehensive-guide-on-item-based-recommendation-systemsd67e40e2b75d<br>
https://towardsdatascience.com/item-based-collaborative-filtering-in-python-91f747200fab

So, here we got an user `sample_test_user_index`, who have rated only 30 items out of 100

In [14]:
df_test.iloc[sample_test_user_index]

1      99.00
2      99.00
3      99.00
4      99.00
5      99.00
       ...  
96     -7.96
97     99.00
98      3.50
99     99.00
100     7.96
Name: 20002, Length: 100, dtype: float64

Now, the main logic of Nearest Neighbour approach of collaborative filtering is that, say, we have an item **P**, whose rating is missing, then
* For reach item, find mean (ignore the missing items so, mean = valid sum / valid ratings count )
* Now, subtract the mean from the valid ratings (missing ratings should remain 0)
* Next compute cosine similarity between **P** (the item for which we are finding ratings) and all other item columns.
* Now choose a value K no of values we are going to consider.
* Choose k items having highest cosine similarity which that user have rated.
* our missing rating will be = sum(weight * rating) / sum(weights)

We are initiaizing dataframe as `df_train_sim` for Nearest Neighbour similarity with all missing ratings = 0

In [15]:
df_train_sim = df_train.copy()
df_train_sim.replace(99.0, 0.0, inplace=True)
df_train_sim

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,6.75,8.64,4.42,7.43,4.56,-0.97,4.66,-0.68,3.30,-1.21,0.87,8.64,8.35,9.17,0.05,7.57,4.71,0.87,-0.39,6.99,6.50,-0.92,7.14,9.03,-1.80,0.73,7.09,3.40,-0.87,7.91,...,-6.70,-3.35,-9.03,4.47,4.08,-3.83,8.74,1.12,0.78,7.52,-5.00,2.77,8.30,7.77,7.33,6.21,7.72,8.98,8.64,8.20,3.93,4.85,4.85,6.07,8.98,4.51,-0.05,3.69,4.56,0.58,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
1,0.00,0.00,0.00,0.00,9.03,9.27,9.03,9.27,0.00,0.00,7.33,7.57,9.37,6.17,-6.36,-6.89,-7.86,9.03,9.03,9.03,7.28,0.00,8.25,0.00,0.00,7.48,7.28,7.28,8.93,0.00,6.17,7.28,0.00,0.00,8.98,7.33,0.00,6.17,9.08,7.33,...,6.46,7.28,0.00,0.00,7.04,7.28,0.00,7.28,8.25,0.00,0.00,0.00,0.00,0.00,0.00,8.93,0.00,0.00,0.00,9.08,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,9.03,0.00,0.00,0.00,9.08,0.00,0.00,0.00,0.00,0.00,0.00
2,0.00,8.35,0.00,0.00,1.80,8.16,-2.82,6.21,0.00,1.84,7.33,6.60,6.31,8.11,-7.23,-6.65,1.17,-6.60,-3.64,-2.09,5.34,0.00,0.00,0.00,0.00,2.91,3.93,6.75,6.60,0.00,6.65,-6.12,0.00,7.57,6.21,6.65,0.00,-8.30,7.18,2.82,...,0.00,-3.69,0.00,0.00,7.82,0.24,0.00,7.28,-2.33,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.63,0.00,0.00,-2.33,0.00,0.00,0.00,0.00,0.00,0.53,0.00,0.00,0.00,0.00,0.00,0.00
3,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,8.25,6.84,-3.93,7.23,-2.33,-9.66,2.72,-1.36,2.57,4.51,8.20,6.12,8.30,-1.26,7.77,1.89,-1.17,5.68,8.45,4.61,8.06,-9.47,7.28,5.68,2.48,3.20,-1.26,6.80,4.51,2.48,...,7.38,6.17,4.71,-2.28,7.38,4.56,7.14,4.22,3.01,3.83,0.00,0.00,0.00,0.00,0.00,0.00,4.13,0.00,0.00,0.00,5.24,5.92,0.87,7.28,3.93,-0.63,6.31,4.71,2.82,2.96,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60
4,-6.17,-3.54,0.44,-8.50,-7.09,-4.32,-8.69,-0.87,-6.65,-1.80,-6.80,-5.73,-5.00,-8.59,0.49,-8.93,-3.69,-2.18,-2.28,-6.12,-3.01,-0.58,-2.38,-7.77,-2.23,-2.28,-5.24,-5.53,-0.68,-6.94,-3.93,-0.34,-6.60,-5.49,-1.55,-0.49,-6.21,-8.06,-5.39,-7.04,...,-6.65,-2.86,-4.61,-9.42,-7.91,-9.81,-9.42,-8.98,-1.94,-6.99,-9.66,-5.19,-5.00,-4.42,-7.28,-6.50,-4.32,-6.94,-9.85,0.73,-6.21,-2.86,-7.62,-6.12,-0.29,-1.41,-3.93,-1.94,-5.68,-4.71,-3.54,-6.89,-0.68,-2.96,-2.18,-3.35,0.05,-9.08,-5.05,-3.45
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19995,0.00,0.00,0.00,0.00,-7.86,0.00,-9.81,6.21,0.00,0.00,0.00,0.00,-7.67,0.00,-7.86,-0.39,-7.33,6.12,-8.50,-0.39,-8.93,0.00,-6.94,0.00,0.00,-6.89,3.59,3.35,-7.82,0.00,-7.82,6.41,0.00,-6.89,3.64,-7.48,0.00,0.00,-6.89,0.00,...,-4.66,-0.39,0.00,0.00,-0.58,-7.62,0.00,-6.80,1.60,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.73,0.00,0.00,0.00,0.00,0.00,-0.49,0.00,0.00,0.00,4.22,0.00
19996,7.57,8.93,1.55,3.06,0.49,2.18,8.98,1.60,-0.49,8.79,6.94,9.03,-0.44,8.79,5.68,-1.07,-0.24,8.25,2.23,2.23,8.59,8.79,8.93,8.93,3.93,8.79,9.03,-1.02,9.13,-0.24,8.01,6.02,4.03,8.83,8.20,8.35,9.13,8.88,1.50,-1.31,...,8.54,-0.63,3.79,-0.49,5.73,-1.02,-0.10,4.42,7.91,3.54,8.59,6.12,0.10,-6.36,6.41,9.03,2.14,2.14,2.14,2.09,8.88,1.07,8.88,9.22,3.35,3.35,8.30,5.63,8.93,2.62,8.83,8.83,8.83,0.00,-1.31,9.03,9.08,-0.78,-2.72,2.52
19997,8.93,-9.17,-4.66,-9.71,8.83,8.88,8.35,8.54,4.85,3.79,1.31,4.66,-4.42,5.58,-2.86,-2.57,2.04,7.52,1.50,8.98,1.26,3.83,5.63,4.85,9.03,5.68,8.74,9.17,9.17,0.39,2.09,-0.83,-9.56,6.21,8.93,8.40,2.91,5.73,8.88,8.83,...,7.82,8.69,-8.35,0.87,3.20,8.45,2.96,8.79,3.45,4.85,5.19,6.75,8.98,-9.13,-9.71,-9.47,8.74,7.33,4.56,8.30,5.53,8.88,8.69,6.94,8.88,4.37,9.03,-9.32,5.39,-0.19,8.69,8.69,5.24,8.88,-9.03,8.35,8.79,0.39,9.13,4.

Here, we are finding mean for every item. We are only considering the valid ratings.

In [16]:
df_train_sim_mean = df_train_sim.sum() / (df_train < 99).sum()
df_train_sim_mean

1      0.925751
2      0.199029
3      0.318721
4     -1.448526
5      0.409909
         ...   
96     1.568010
97     1.709530
98     0.688166
99    -0.008948
100    1.335644
Length: 100, dtype: float64

Now, we are subtracting mean from every column keeping the missing values = 0.0

In [17]:
df_train_sim = (df_train_sim - df_train_sim_mean)

for i in range(100):
  df_train_sim.loc[df_train[str(i+1)] == 99.0, str(i+1)] = 0.0

df_train_sim

,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,...,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100
0,3.154249,-0.489029,6.041279,5.818526,-2.789909,-11.312838,-0.281215,-4.775605,9.367522,7.848915,4.914435,7.218692,6.136872,6.041136,6.193627,2.115287,5.764052,-0.093889,3.14187,-0.333329,-1.178794,7.793307,8.188554,10.809559,-0.340256,6.260842,1.501288,-0.688181,-3.359779,7.382599,4.297784,-4.086031,8.514993,8.205759,-4.810398,-2.628208,8.425835,2.052247,-1.92758,6.94404,...,-9.242408,-6.37722,-9.309149,5.094489,1.7973,-6.425872,9.574739,-1.587317,-1.793723,7.130479,-4.412942,0.061241,7.224144,9.356223,7.63278,3.69896,7.035953,7.25738,8.570411,7.140385,2.154541,3.923209,2.918463,5.393099,8.022289,4.222607,-1.953699,1.59227,0.949511,-0.048698,0.794518,-6.147113,-2.781408,6.678426,-1.229414,-3.70801,1.35047,-0.348166,-4.311052,-0.265644
1,0.000000,0.000000,0.000000,0.000000,8.620091,7.617162,9.478785,9.834395,0.000000,0.000000,5.494435,6.148692,11.086872,4.781136,-4.726373,-3.804713,-6.755948,9.616111,8.87187,9.906671,5.231206,0.000000,8.088554,0.000000,0.000000,6.170842,4.071288,5.721819,5.960221,0.000000,3.967784,4.113969,0.000000,0.000000,5.969602,3.971792,0.000000,4.822247,8.02242,6.36404,...,3.917592,4.25278,0.000000,0.000000,4.7573,4.684128,0.000000,4.572683,5.676277,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,6.41896,0.000000,0.00000,0.000000,8.020385,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,8.401302,0.000000,0.000000,0.000000,7.898426,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000
2,0.000000,8.150971,0.000000,0.000000,1.390091,6.507162,-2.371215,6.774395,0.000000,0.468915,5.494435,5.178692,8.026872,6.721136,-5.596373,-3.564713,2.274052,-6.013889,-3.79813,-1.213329,3.291206,0.000000,0.000000,0.000000,0.000000,1.600842,0.721288,5.191819,3.630221,0.000000,4.447784,-9.286031,0.000000,6.745759,3.199602,3.291792,0.000000,-9.647753,6.12242,1.85404,...,-2.542408,-6.71722,0.000000,0.000000,5.5373,-2.355872,0.000000,4.572683,-4.903723,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-0.327711,0.000000,0.000000,-4.42773,0.000000,0.000000,0.000000,0.000000,0.000000,-0.651574,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000
3,7.574249,4.410971,-4.488721,-3.941474,0.950091,-0.052838,7.488785,5.174395,0.047522,4.358915,6.414435,5.418692,-2.213128,5.841136,-0.696373,-6.574713,3.824052,-0.773889,2.41187,5.386671,6.151206,5.273307,8.138554,0.379559,7.379744,0.580842,-4.378712,4.121819,5.480221,5.002599,5.857784,-12.636031,8.654993,4.855759,-0.530398,-0.158208,0.075835,5.452247,3.45242,1.51404,...,4.837592,3.14278,4.430851,-1.655511,5.0973,1.964128,7.974739,1.512683,0.436277,3.440479,0.000000,0.000000,0.000000,0.000000,0.00000,0.00000,3.445953,0.00000,0.000000,0.000000,3.464541,4.993209,-1.061537,6.603099,2.972289,-0.917393,4.406301,2.61227,-0.790489,2.331302,3.164518,4.382887,1.778592,4.008426,4.690586,-0.01801,1.40047,5.861834,1.808948,0.264356
4,-7.095751,-3.739029,0.121279,-7.051474,-7.499909,-5.972838,-8.241215,-0.305605,-6.162478,-3.171085,-8.635565,-7.151308,-3.283128,-9.978864,2.123627,-5.844713,-2.585948,-1.593889,-2.43813,-5.243329,-5.058794,-1.426693,-2.541446,-6.130441,-2.620256,-3.589158,-8.448712,-7.088181,-3.649779,-6.547401,-6.132216,-3.506031,-5.225007,-6.314241,-4.560398,-3.848208,-4.874165,-9.407753,-6.44758,-8.00596,...,-9.192408,-5.88722,-4.889149,-8.795511,-10.1927,-12.405872,-8.585261,-11.687317,-4.513723,-7.379521,-9.072942,-7.898759,-6.075856,-2.833777,-6.97722,-9.01104,-5.004047,-8.66262,-9.919589,-0.329615,-7.985459,-3.786791,-9.551537,-6.796901,-1.247711,-1.697393,-5.833699,-4.03773,-9.290489,-5.338698,-5.565482,-8.087113,-3.171408,-4.141574,-3.219414,-4.91801,-1.65953,-9.768166,-5.041052,-4.785644
...,...,...,...,...,...,...,...,...,...,...

Here, actually, we are trying to fing ratings of the user `sample_test_user_index` for item no `'1'`<br>
So we are finding cosine similarity of column '1' and all other columns.

In [18]:
cos = []

for i in range(100):
  x = (df_train_sim['1'] * df_train_sim[str(i+1)]).sum() / (np.linalg.norm(df_train_sim['1'], 2) * np.linalg.norm(df_train_sim[str(i+1)], 2))
  cos.append([x, i])
cos.sort(reverse=True)

In [29]:
for i in cos: print(f'Similarity of column {i[1]}\t->  {i[0]}')

Similarity of column 0	->  0.9999999999999998
Similarity of column 1	->  0.3671089932718648
Similarity of column 2	->  0.35603433588777167
Similarity of column 63	->  0.3398833944848428
Similarity of column 36	->  0.32702171895385185
Similarity of column 8	->  0.32185094480715803
Similarity of column 43	->  0.3159337228453941
Similarity of column 9	->  0.3050271344943007
Similarity of column 32	->  0.30324447841921726
Similarity of column 59	->  0.29625733214213124
Similarity of column 58	->  0.2861167801265506
Similarity of column 3	->  0.2844333315625499
Similarity of column 23	->  0.2759229559321425
Similarity of column 40	->  0.27570806128484715
Similarity of column 56	->  0.2754567715968707
Similarity of column 42	->  0.27541522597407
Similarity of column 24	->  0.2752333690652218
Similarity of column 22	->  0.26291392520432827
Similarity of column 54	->  0.2622619422614894
Similarity of column 37	->  0.2621144779746263
Similarity of column 29	->  0.2554346293592817
Similarity of 

Finally we are fetching only 10 most similar values out of 30 items with known rating from the user `sample_test_user_index`

In [24]:
choosen_weights = []

for i in cos:
  if df_test.iloc[sample_test_user_index, i[1]] != 99.0:
    choosen_weights.append(i)
  if len(choosen_weights) == 10: break

choosen_weights

[[0.3159337228453941, 43],
 [0.30324447841921726, 32],
 [0.2861167801265506, 58],
 [0.2759229559321425, 23],
 [0.2752333690652218, 24],
 [0.24845932871437595, 66],
 [0.2439878414396036, 39],
 [0.18972453994312008, 47],
 [0.18723468737086338, 30],
 [0.163144530717111, 93]]

So, finally we can calculate the missing rating from the ratings of most similar 10 items as

In [25]:
sum_value, sum_weights = 0, 0

for i in choosen_weights:
  sum_value += df_test.iloc[sample_test_user_index, i[1]] * i[0]
  sum_weights += i[0]

missing_rating = round(sum_value / sum_weights, 6)
missing_rating

3.175504

### **Question 3 :** Matrix Factorization approach (Latent Factors) (20 marks)

https://pypi.org/project/PyMF/

https://github.com/gbolmier/funk-svd

https://nimfa.biolab.si/nimfa.examples.recommendations.html

Use any of the above libraries to factorize the rating matrix R, into a product of 2 matrices; P and Q. P contains the latent user-interests and Q the item latent factors. You can use the training set to learn the parameters and apply it to the test set to make the predictions of ratings.
By limiting the size of P and Q, you can essentially impose the low-rank constraint. Compute the rank of the matrix PQ produced by different approaches. Plot the quality of your results vs. rank.

#### **Methode : 1** Factorizing using Gradient Descent

In [65]:
def matrix_factorization(R, P, Q, K, steps=5, alpha=0.0002, beta=0.02):
    '''
    R: rating matrix
    P: |U| * K (User features matrix)
    Q: |D| * K (Item features matrix)
    K: latent features
    steps: iterations
    alpha: learning rate
    beta: regularization parameter'''
    Q = Q.T

    for step in range(steps):
        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    # calculate error
                    eij = R[i][j] - numpy.dot(P[i,:],Q[:,j])
                    for k in range(K):
                        # calculate gradient with a and beta parameter
                        P[i][k] = P[i][k] + alpha * (2 * eij * Q[k][j] - beta * P[i][k])
                        Q[k][j] = Q[k][j] + alpha * (2 * eij * P[i][k] - beta * Q[k][j])

        eR = numpy.dot(P,Q)
        e = 0

        for i in range(len(R)):
            for j in range(len(R[i])):
                if R[i][j] > 0:
                    e = e + pow(R[i][j] - numpy.dot(P[i,:],Q[:,j]), 2)
                    for k in range(K):
                        e = e + (beta/2) * (pow(P[i][k],2) + pow(Q[k][j],2))
        # 0.001: local minimum
        if e < 0.001: break
    return P, Q.T

* Now, we are adding 10 with all ratings to eliminate negative ratings and shift the range from [-10, 10] to [0, 20]
* Next, we are replacing 109.0  (actually 99.0 + 10) by 0

In [66]:
test_dataset = df_train.copy()
test_dataset = test_dataset + 10
test_dataset.replace(109.0, 0.0, inplace=True)

R = numpy.array(test_dataset)
N = len(R)
M = len(R[0])
K = 75
 
P = numpy.random.rand(N,K)
Q = numpy.random.rand(M,K)

nP, nQ = matrix_factorization(R, P, Q, K)

In [69]:
print(nP.shape)
print(nQ.shape)

nP = np.matrix(nP)
nQ = np.matrix(nQ)

nR = nP * nQ.T

print(nR.shape)

(20000, 75)
(100, 75)
(20000, 100)


In [70]:
np.linalg.matrix_rank(nR)

75

In [71]:
nR = nR - 10
nR

matrix([[ 1.51200442,  0.32552597,  0.73846278, ...,  1.21744501,
          1.6539108 ,  1.36333105],
        [ 4.66847361,  3.88992457,  3.35492021, ...,  4.52554409,
          3.52772666,  4.64426033],
        [ 1.0790352 ,  0.39458948, -0.80565132, ...,  0.86165621,
          0.86336157,  1.10890733],
        ...,
        [ 4.08460357,  3.56059791,  3.52051666, ...,  3.75905319,
          3.97659089,  4.50572533],
        [ 3.95230781,  3.54330303,  2.14688586, ...,  2.31753409,
          2.19603411,  3.5716517 ],
        [ 3.7322829 ,  1.44510356,  1.87982862, ...,  2.88096733,
          2.82587734,  2.51335973]])

#### **Method 2 :** SVD

In [58]:
from scipy.linalg import svd

test_dataset = df_train.copy()
test_dataset.replace(109.0, 0.0, inplace=True)
A = np.array(test_dataset)

U, s, VT = svd(A)

print(s)

[64238.76866908 22421.83129421  9204.07429354  7145.18889895
  5002.49177949  4720.26108832  4522.74716779  4508.12896674
  3910.41963267  3857.6320069   3733.6759063   3637.58458924
  3594.90103666  3544.45862742  3510.84075684  3462.85517577
  3405.86507578  3376.71555477  3340.79141042  3330.04451825
  3301.98030295  3277.47881203  3272.59018221  3257.32120657
  3241.08014805  3228.5976997   3209.75374129  3205.91615074
  3190.38694928  3185.60428175  3168.88657503  3149.30002978
  3126.65825039  3112.81918976  3093.92270818  3079.71836005
  3076.25344863  3063.55740685  3051.72139164  3043.48601056
  3018.63347155  3010.38592279  2991.27799375  2977.53930257
  2957.99652129  2956.12841817  2922.73139903  2898.86664545
  2889.80564603  2878.22188185  2841.01664106  2808.79675853
  2784.74168968  2727.29406719  2705.84498492  2647.56417452
  2615.55852781  2548.60361628  2525.86948722  2431.96934358
  2395.25329778  2342.53505873  2306.51593214  2241.05739956
  2199.0770904   2150.66

* First, to minimize rank, we are considering only 78 features of the matrix
* Next, we calculate sqrt of the matrix sigma say sqrt_sigma
* Now, to get P = U * sqrt_sigma and Q = sqrt_sigma * VT

In [59]:
s_mod_U = np.diag(s[:78]) ** 0.5
s_mod_U = np.concatenate((s_mod_U, np.zeros((20000-78, 78))))
s_mod_U = np.matrix(s_mod_U)
s_mod_U.shape

(20000, 78)

In [60]:
P = U * s_mod_U
P.shape

(20000, 78)

In [61]:
s_mod_V = np.diag(s[:78]) ** 0.5
s_mod_V = np.concatenate((s_mod_V, np.zeros((78, 100 - 78))), axis=1)
s_mod_V = np.matrix(s_mod_V)
s_mod_V.shape

(78, 100)

In [62]:
Q = s_mod_V * VT
Q.shape

(78, 100)

In [63]:
R = P * Q
R.shape

(20000, 100)

In [64]:
np.linalg.matrix_rank(R)

78